In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

path = "../../../dane/8CPU_20RAM/3600s/all_merged.csv"

In [2]:
# Załaduj dane
df = pd.read_csv(path)

In [3]:
for col in ['replicaId', 'endpointUrl_methods']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [4]:
# Dodajemy skalowanie dla wybranych kolumn
features_to_scale = ['queueSizeForward_methods', 'queueSizeBack_methods',
                     'cpuUsage_stock', 'memoryUsage_stock',
                     'applicationTime_trading', 'databaseTime_trading',
                     'numberOfSellOffers_trading', 'numberOfBuyOffers_trading']
scaler = StandardScaler()
for col in features_to_scale:
    df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

In [5]:
# Wybór cech
features = [
            # 'timestamp',
            # 'apiTime_methods',
            # 'applicationTime_methods',
            # 'databaseTime_methods',
            'endpointUrl_methods',
            # 'queueSizeForward_methods',
            # 'queueSizeBack_methods',
            'cpuUsage_stock',
            'memoryUsage_stock',
            # 'applicationTime_trading',
            # 'databaseTime_trading',
            # 'numberOfSellOffers_trading',
            # 'numberOfBuyOffers_trading',
            # 'cpuUsage_traffic',
            # 'memoryUsage_traffic',
            # 'replicaId'
            ]
df_features = df[features]

# Kodowanie pudełek
le = LabelEncoder()
df['test'] = le.fit_transform(df['test'])

# Kopiowanie danych
df_encoded = df_features.copy()
df_encoded['test'] = df['test']

In [6]:
# Stwórz puste listy do przechowywania danych treningowych i testowych
X_train = []
X_test = []
y_train = []
y_test = []

In [7]:
def create_windows(X, y, window_size, step_size):
    X_windows = []
    y_windows = []

    # Przesuń okno po danych
    for i in range(0, len(X) - window_size, step_size):
        # Utwórz okno danych
        X_window = X.iloc[i:i + window_size]
        # Utwórz etykietę dla okna (etykieta ostatniej obserwacji w oknie)
        y_window = y.iloc[i + window_size]

        X_windows.append(X_window.values)
        y_windows.append(y_window)

    return np.array(X_windows), np.array(y_windows)

In [8]:
window_size = 700
step_size = 100
# Dla każdego unikalnego testu
for box in df_encoded['test'].unique():
    # Wybierz tylko rekordy dla tego testu
    box_data = df_encoded[df_encoded['test'] == box]
    
    # Oblicz punkt podziału (80% danych)
    split_point = int(len(box_data) * 0.8)
    
    # Dodaj pierwsze 80% rekordów do danych treningowych
    X_train_box = box_data.drop('test', axis=1).iloc[:split_point]
    y_train_box = box_data['test'].iloc[:split_point]
    
    # Dodaj ostatnie 20% rekordów do danych testowych
    X_test_box = box_data.drop('test', axis=1).iloc[split_point:]
    y_test_box = box_data['test'].iloc[split_point:]
    
    # Stwórz okienka dla danych treningowych
    X_train_windows, y_train_windows = create_windows(X_train_box, y_train_box, window_size, step_size)
    
    # Stwórz okienka dla danych testowych
    X_test_windows, y_test_windows = create_windows(X_test_box, y_test_box, window_size, step_size)
    
    # Dodaj okienka do list
    X_train.append(X_train_windows)
    y_train.append(y_train_windows)
    X_test.append(X_test_windows)
    y_test.append(y_test_windows)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from keras.utils import to_categorical

In [10]:
# Połączenie danych treningowych dla wszystkich testów
X_train_combined = np.concatenate(X_train, axis=0)
y_train_combined = np.concatenate(y_train, axis=0)

In [11]:
# Zakodowanie etykiet w formacie one-hot encoding
y_train_combined_encoded = to_categorical(y_train_combined, num_classes=len(df['test'].unique()))

In [12]:
# Trenowanie modelu na połączonym zestawie danych
model = Sequential()    
model.add(GRU(50, return_sequences=False, input_shape=(window_size, X_train_combined.shape[2])))
model.add(Dense(len(df['test'].unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_combined, y_train_combined_encoded, epochs=25, batch_size=32)

Epoch 1/25
2023/2023 [==============================] - 347s 170ms/step - loss: 2.1596 - accuracy: 0.2793
Epoch 2/25
2023/2023 [==============================] - 313s 154ms/step - loss: 1.9085 - accuracy: 0.3576
Epoch 3/25
2023/2023 [==============================] - 290s 144ms/step - loss: 1.8277 - accuracy: 0.3928
Epoch 4/25
2023/2023 [==============================] - 290s 143ms/step - loss: 1.6952 - accuracy: 0.4253
Epoch 5/25
2023/2023 [==============================] - 288s 142ms/step - loss: 1.5619 - accuracy: 0.4655
Epoch 6/25
2023/2023 [==============================] - 289s 143ms/step - loss: 1.4353 - accuracy: 0.5042
Epoch 7/25
2023/2023 [==============================] - 291s 144ms/step - loss: 1.3400 - accuracy: 0.5335
Epoch 8/25
2023/2023 [==============================] - 288s 143ms/step - loss: 1.2443 - accuracy: 0.5714
Epoch 9/25
2023/2023 [==============================] - 291s 144ms/step - loss: 1.1274 - accuracy: 0.6107
Epoch 10/25
2023/2023 [=======================

In [13]:
for i in range(len(X_test)):
    y_test_encoded = to_categorical(y_test[i], num_classes=len(df['test'].unique()))
    loss, accuracy = model.evaluate(X_test[i], y_test_encoded)
    print(f"Test {i+1}: Loss = {loss}, Accuracy = {accuracy}")

34/34 [==============================] - 1s 32ms/step - loss: 1.1584 - accuracy: 0.6832
Test 1: Loss = 1.1584311723709106, Accuracy = 0.6832239627838135
34/34 [==============================] - 1s 32ms/step - loss: 2.2652 - accuracy: 0.3318
Test 2: Loss = 2.2652127742767334, Accuracy = 0.33176690340042114
31/31 [==============================] - 1s 32ms/step - loss: 2.2506 - accuracy: 0.1455
Test 3: Loss = 2.2505533695220947, Accuracy = 0.14547304809093475
33/33 [==============================] - 1s 32ms/step - loss: 3.4734 - accuracy: 0.0276
Test 4: Loss = 3.4733963012695312, Accuracy = 0.027566539123654366
26/26 [==============================] - 1s 32ms/step - loss: 0.2092 - accuracy: 0.9334
Test 5: Loss = 0.2091863453388214, Accuracy = 0.9334140419960022
29/29 [==============================] - 1s 32ms/step - loss: 0.8426 - accuracy: 0.8393
Test 6: Loss = 0.8425816893577576, Accuracy = 0.8392664790153503
16/16 [==============================] - 1s 32ms/step - loss: 0.1606 - accurac